In [1]:
!pip install filterpy
import json
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import leastsq, curve_fit, least_squares
from collections import deque
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import random
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import pandas as pd
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=bd81e7a6b6e1303eb9a09f33614da8194e25c0383099da3054a6cb45dec959ea
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy


In [2]:
def load_trajectory_data(json_file, start=None, end=None):
    # Load JSON data
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Extract and convert values
    x_vals = [point['x'] / 1000 for point in data]  # Convert mm to m
    y_vals = [point['y'] / 1000 for point in data]
    z_vals = [point['z'] / 1000 for point in data]
    t_vals = [point['t'] / 1_000_000 for point in data]  # Convert µs to s

    # Normalize time so the first timestamp is at t = 0
    t_start = t_vals[0]
    t_vals = [t - t_start for t in t_vals]

    # Apply slicing only if start or end is provided
    return (
        x_vals[start:end] if start is not None or end is not None else x_vals,
        y_vals[start:end] if start is not None or end is not None else y_vals,
        z_vals[start:end] if start is not None or end is not None else z_vals,
        t_vals[start:end] if start is not None or end is not None else t_vals,
    )

def plot_3d_scatter(x_vals, y_vals, z_vals):
    # Create a pandas DataFrame from the data
    df = pd.DataFrame({'x': x_vals, 'y': y_vals, 'z': z_vals, 'index': list(range(len(x_vals)))})

    # Create a 3D scatter plot using the DataFrame
    fig = px.scatter_3d(
        df,  # Pass the DataFrame
        x='x', y='y', z='z',
        labels={'x': 'x (meters)', 'y': 'y (meters)', 'z': 'z (meters)'},
        title="3D Coordinate Plot",
        hover_data={'x': False, 'y': False, 'z': False, 'index': True}  # Use column names
    )

    # Add the index as text labels for each point
    fig.update_traces(
        text=[f"Idx: {i}" for i in range(len(x_vals))],  # Add index as text labels
        marker=dict(
            size=10,  # Circle size
            color='rgba(0, 0, 0, 0)',  # Transparent fill
            line=dict(color='blue', width=2)  # Outline color and width
        )
    )

    # Set axis ranges dynamically
    range_x = [min(x_vals), max(x_vals)]
    range_y = [min(y_vals), max(y_vals)]
    range_z = [min(z_vals), max(z_vals)]

    # Update the layout to set the y-axis as the vertical axis
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='x (meters)', range=range_x),
            yaxis=dict(title='y (meters)', range=range_y),
            zaxis=dict(title='z (meters)', range=range_z),
            aspectmode="data",
        ),
        scene_camera=dict(
            up=dict(x=0, y=1, z=0)  # Set y-axis as the vertical axis
        )
    )

    # Show the plot
    return fig

def add_point():
    # Add a single red point to the figure
    fig.add_scatter3d(
        x=[x], y=[y], z=[z],
        mode='markers',
        marker=dict(size=10, color='red'),
        name='Interception Point'
    )

    return fig

def estimate_trajectory(x_vals, y_vals, z_vals, t_vals, gravity_vector):
    # Unpack gravity vector
    ax, ay, az = gravity_vector

    # Model function with known accelerations (ax, ay, az)
    def model(t, x0, v0, a):
        return x0 + v0 * t + 0.5 * a * t**2

    # Fit for initial position (x0) and velocity (v0), using known acceleration (ax, ay, az)
    x_params, _ = curve_fit(lambda t, x0, v0: model(t, x0, v0, ax), t_vals, x_vals, p0=[x_vals[0], x_vals[1] - x_vals[0]])
    y_params, _ = curve_fit(lambda t, y0, v0: model(t, y0, v0, ay), t_vals, y_vals, p0=[y_vals[0], y_vals[1] - y_vals[0]])
    z_params, _ = curve_fit(lambda t, z0, v0: model(t, z0, v0, az), t_vals, z_vals, p0=[z_vals[0], z_vals[1] - z_vals[0]])

    # Return initial positions and velocities, accelerations are known constants
    initial_position = (x_params[0], y_params[0], z_params[0])
    initial_velocity = (x_params[1], y_params[1], z_params[1])

    return {
        'initial_position': initial_position,
        'initial_velocity': initial_velocity,
        'acceleration': (ax, ay, az)
    }

def parse_prediction_data(data):
    mapping = {
        "Initial Position": "initial_position",
        "Initial Velocity": "initial_velocity",
        "Acceleration": "acceleration",
        "Interception point": "intercept"
    }

    result = {}
    lines = data.split("\n")
    for line in lines:
        match = re.match(r"(.*?):?\s+([-\d\.]+)\s+([-\d\.]+)\s+([-\d\.]+)", line)
        if match:
            key, x, y, z = match.groups()
            if key in mapping:
                result[mapping[key]] = (float(x), float(y), float(z))

    return result

# Function to compute x, y, z positions with time offset
def x(t, t0, x0, vx0, _ax0):
    return x0 + vx0 * (t - t0) + 0.5 * _ax0 * np.power(t - t0, 2)


# Function to plot predictions
def plot_prediction_c(idx, json_file, prediction_data, start=None, end=None):
    # Load trajectory data
    x_vals, y_vals, z_vals, t_vals = load_trajectory_data(json_file, start, end)
    params = parse_prediction_data(prediction_data)

    colors = [
        'red', 'green', 'purple', 'orange', 'yellow',
        'pink', 'brown', 'black', 'gray', 'lightblue', 'lightgreen',
        'lightpink', 'cyan', 'magenta', 'gold', 'lime', 'teal',
        'navy', 'maroon', 'olive', 'coral', 'indigo', 'violet'
    ]

    fig = go.Figure()

    fig.update_layout(
        title=f"Predicted Projectile Motion, Iteration: {idx}",
        scene=dict(
            xaxis_title="x (meters)",
            yaxis_title="y (meters)",
            zaxis_title="z (meters)",
            aspectmode="data"
        ),
        scene_camera=dict(
            up=dict(x=0, y=1, z=0)  # Set y-axis as the vertical axis
        )
    )

    # Points used in prediction
    fig.add_scatter3d(
        x=x_vals[:idx],
        y=y_vals[:idx],
        z=z_vals[:idx],
        mode='markers',
        name='Points Used in Prediction',
        marker=dict(
            size=10,
            color='blue',
            line=dict(color='blue', width=2)
        )
    )

    # Actual trajectory
    fig.add_scatter3d(
        x=x_vals[idx:],
        y=y_vals[idx:],
        z=z_vals[idx:],
        mode='markers',
        name='Actual Trajectory',
        marker=dict(
            size=10,
            color='rgba(0, 0, 0, 0)',
            line=dict(color='green', width=2)
        )
    )

    # Predicted trajectory
    # _t0, _x0, _y0, _z0, _vx0, _vy0, _vz0 = guesses[idx]
    _t0 = 0
    start = None
    end = idx + 1
    _x0, _y0, _z0 = params['initial_position']
    _vx0, _vy0, _vz0 = params['initial_velocity']
    _ax0, _ay0, _az0 = params['acceleration']


    fig.add_scatter3d(
        x=x(np.array(t_vals[idx:]), _t0, _x0, _vx0, _ax0),
        y=x(np.array(t_vals[idx:]), _t0, _y0, _vy0, _ay0),
        z=x(np.array(t_vals[idx:]), _t0, _z0, _vz0, _az0),
        mode='markers',
        name='Predicted Trajectory',
        marker=dict(
            size=10,
            color='red',
            line=dict(color='red', width=2)
        )
    )

    # xyz_pred = [x(t_vals[-1], _t0, _x0, _vx0, _ax0), x(t_vals[-1], _t0, _y0, _vy0, _ay0), x(t_vals[-1], _t0, _z0, _vz0, _az0)]
    # xyz_actual = [x_vals[-1], y_vals[-1], z_vals[-1]]
    # error = [abs(a - p) for a, p in zip(xyz_actual, xyz_pred)]
    # total_error = np.linalg.norm(error)
    # print(f"Absolute error in final prediction vs final trajectory point: {total_error:.2f}m")

    # Show the plot
    fig.show()

gravity_vector = [-0.5839661, 10.99642713, 1.39555507]

def plot_prediction_python(idx, json_file, start=None, end=None):
    x_vals, y_vals, z_vals, t_vals = load_trajectory_data(json_file, start, end)
    colors = [
        'red', 'green', 'purple', 'orange', 'yellow',
        'pink', 'brown', 'black', 'gray', 'lightblue', 'lightgreen',
        'lightpink', 'cyan', 'magenta', 'gold', 'lime', 'teal',
        'navy', 'maroon', 'olive', 'coral', 'indigo', 'violet'
    ]

    fig = go.Figure()

    fig.update_layout(
        title=f"Predicted Projectile Motion, Iteration: {idx}",
        scene=dict(
            xaxis_title="x (meters)",
            yaxis_title="y (meters)",
            zaxis_title="z (meters)",
            aspectmode="data"
        ),
        scene_camera=dict(
            up=dict(x=0, y=1, z=0)  # Set y-axis as the vertical axis
        )
    )

    # Points used in prediction
    fig.add_scatter3d(
        x=x_vals[:idx],
        y=y_vals[:idx],
        z=z_vals[:idx],
        mode='markers',
        name='Points Used in Prediction',
        marker=dict(
            size=10,
            color='blue',
            line=dict(color='blue', width=2)
        )
    )

    # Actual trajectory
    fig.add_scatter3d(
        x=x_vals[idx:],
        y=y_vals[idx:],
        z=z_vals[idx:],
        mode='markers',
        name='Actual Trajectory',
        marker=dict(
            size=10,
            color='rgba(0, 0, 0, 0)',
            line=dict(color='green', width=2)
        )
    )

    # Predicted trajectory
    # _t0, _x0, _y0, _z0, _vx0, _vy0, _vz0 = guesses[idx]
    _t0 = 0
    start = None
    end = idx + 1
    params = estimate_trajectory(x_vals[start:end], y_vals[start:end], z_vals[start:end], t_vals[start:end], gravity_vector)
    _x0, _y0, _z0 = params['initial_position']
    _vx0, _vy0, _vz0 = params['initial_velocity']
    _ax0, _ay0, _az0 = params['acceleration']


    fig.add_scatter3d(
        x=x(np.array(t_vals[idx:]), _t0, _x0, _vx0, _ax0),
        y=x(np.array(t_vals[idx:]), _t0, _y0, _vy0, _ay0),
        z=x(np.array(t_vals[idx:]), _t0, _z0, _vz0, _az0),
        mode='markers',
        name='Predicted Trajectory',
        marker=dict(
            size=10,
            color='red',
            line=dict(color='red', width=2)
        )
    )

    fig.show()


In [ ]:
plot_prediction_c(15, "test_prediction_01.json", """Initial Position:  -0.883812 0.47686 3.30701
Initial Velocity:  3.42489 -3.47421 -3.0898
Acceleration -0.583966 10.9964 1.39556
Interception point: 0.755639 0.114308 1.93655""")

In [ ]:
plot_prediction_python(15, "test_prediction_01.json")

In [ ]:
plot_prediction_c(15, "test_prediction_03.json", """Initial Position:  -0.845183 0.447246 3.2483
Initial Velocity:  2.96828 -3.17491 -2.53567
Acceleration -0.583966 10.9964 1.39556
Interception point: 0.565959 0.234347 2.15491""")

In [ ]:
plot_prediction_c(15, "test_prediction_04.json", """Initial Position:  -0.926209 0.402927 3.3394
Initial Velocity:  3.03002 -3.31 -3.39513
Acceleration -0.583966 10.9964 1.39556
Interception point: 0.515806 0.122481 1.81629""")

In [ ]:
plot_prediction_python(15, "test_prediction_04.json")

In [22]:
def model(t, t0, x0, vx0, _ax0):
    return x0 + vx0 * (t - t0) + 0.5 * _ax0 * np.power(t - t0, 2)

def plot_interception(idx, interception_time, json_file, prediction_data, start=None, end=None):
    """
    Plot trajectory with interception point highlighted at given time

    Parameters:
    -----------
    idx : int
        Number of points used for prediction
    json_file : str
        Path to JSON file with trajectory data
    prediction_data : str
        Prediction data as string
    interception_time : float
        Time at which interception occurs
    start : int, optional
        Start index for data
    end : int, optional
        End index for data
    """
    # Load trajectory data
    x_vals, y_vals, z_vals, t_vals = load_trajectory_data(json_file, start, end)
    params = parse_prediction_data(prediction_data)

    # Extract parameters
    _t0 = 0
    _x0, _y0, _z0 = params['initial_position']
    _vx0, _vy0, _vz0 = params['initial_velocity']
    _ax0, _ay0, _az0 = params['acceleration']

    # Calculate interception point coordinates
    intercept_x = model(interception_time, _t0, _x0, _vx0, _ax0)
    intercept_y = model(interception_time, _t0, _y0, _vy0, _ay0)
    intercept_z = model(interception_time, _t0, _z0, _vz0, _az0)

    # Find the closest data point to the interception time
    closest_idx = np.argmin(np.abs(np.array(t_vals) - interception_time))
    closest_time = t_vals[closest_idx]
    closest_x = x_vals[closest_idx]
    closest_y = y_vals[closest_idx]
    closest_z = z_vals[closest_idx]

    # Create the plot
    fig = go.Figure()

    fig.update_layout(
        title=f"Predicted Trajectory with Interception Point at t={interception_time:.3f}s",
        scene=dict(
            xaxis_title="x (meters)",
            yaxis_title="y (meters)",
            zaxis_title="z (meters)",
            aspectmode="data"
        ),
        scene_camera=dict(
            up=dict(x=0, y=1, z=0)  # Set y-axis as the vertical axis
        )
    )

    # Points used in prediction
    fig.add_scatter3d(
        x=x_vals[:idx],
        y=y_vals[:idx],
        z=z_vals[:idx],
        mode='markers',
        name='Points Used in Prediction',
        marker=dict(
            size=6,
            color='blue',
            line=dict(color='blue', width=1)
        )
    )

    # Actual trajectory
    fig.add_scatter3d(
        x=x_vals[idx:],
        y=y_vals[idx:],
        z=z_vals[idx:],
        mode='markers',
        name='Actual Trajectory',
        marker=dict(
            size=6,
            color='green',
            line=dict(color='green', width=1)
        )
    )

    # Predicted trajectory
    fig.add_scatter3d(
        x=model(np.array(t_vals[idx:]), _t0, _x0, _vx0, _ax0),
        y=model(np.array(t_vals[idx:]), _t0, _y0, _vy0, _ay0),
        z=model(np.array(t_vals[idx:]), _t0, _z0, _vz0, _az0),
        mode='markers',
        name='Predicted Trajectory',
        marker=dict(
            size=6,
            color='red',
            line=dict(color='red', width=1)
        )
    )

    # Add interception point (light pink)
    fig.add_scatter3d(
        x=[intercept_x],
        y=[intercept_y],
        z=[intercept_z],
        mode='markers',
        name=f'Interception Point (t={interception_time:.3f}s)',
        marker=dict(
            size=12,
            color='lightpink',
            symbol='diamond',
            line=dict(color='black', width=1)
        )
    )

    # Add closest time data point (lime)
    fig.add_scatter3d(
        x=[closest_x],
        y=[closest_y],
        z=[closest_z],
        mode='markers',
        name=f'Closest Data Point (t={closest_time:.3f}s)',
        marker=dict(
            size=12,
            color='lime',
            symbol='circle',
            line=dict(color='black', width=1)
        )
    )

    # Print distance between interception point and closest data point
    distance = np.sqrt((intercept_x - closest_x)**2 +
                       (intercept_y - closest_y)**2 +
                       (intercept_z - closest_z)**2)

    print(f"Interception point: ({intercept_x:.3f}, {intercept_y:.3f}, {intercept_z:.3f}) at t={interception_time:.3f}s")
    print(f"Closest data point: ({closest_x:.3f}, {closest_y:.3f}, {closest_z:.3f}) at t={closest_time:.3f}s")
    print(f"Time difference: {abs(interception_time - closest_time):.3f}s")
    print(f"Spatial distance: {distance:.3f}m")
    print(f"Time to intercept: {(interception_time - t_vals[idx]):.3f}ms")

    # Show the plot
    fig.show()

    return {
        "interception_point": (intercept_x, intercept_y, intercept_z),
        "closest_point": (closest_x, closest_y, closest_z),
        "interception_time": interception_time,
        "closest_time": closest_time,
        "distance": distance
    }

In [ ]:
plot_interception(15, 0.5, "test_prediction_04.json", """Initial Position:  -0.926209 0.402927 3.3394
Initial Velocity:  3.03002 -3.31 -3.39513
Acceleration -0.583966 10.9964 1.39556
Interception point: 0.515806 0.122481 1.81629""")

Interception point: (0.516, 0.122, 1.816) at t=0.500s
Closest data point: (0.576, 0.118, 1.721) at t=0.501s
Time difference: 0.001s
Spatial distance: 0.112m


{'interception_point': (np.float64(0.51580525),
  np.float64(0.12247699999999995),
  np.float64(1.81628)),
 'closest_point': (0.5755892944335937, 0.11784009552001953, 1.72113037109375),
 'interception_time': 0.5,
 'closest_time': 0.5014752399993085,
 'distance': np.float64(0.11246814986026683)}

In [ ]:
plot_interception(15, 0.5, "test_prediction_05.json", """Initial Position:  -0.89878 0.4002 3.29274
Initial Velocity:  2.6796 -3.57198 -3.52857
Acceleration -0.583966 10.9964 1.39556
Interception point: 0.368025 -0.0112366 1.7029""")

Interception point: (0.368, -0.011, 1.703) at t=0.500s
Closest data point: (0.397, -0.029, 1.649) at t=0.501s
Time difference: 0.001s
Spatial distance: 0.064m
Time to intercept: 0.280ms


{'interception_point': (np.float64(0.36802425000000005),
  np.float64(-0.011240000000000139),
  np.float64(1.7028999999999996)),
 'closest_point': (0.39722457885742185,
  -0.029284751892089844,
  1.648747802734375),
 'interception_time': 0.5,
 'closest_time': 0.5014749760002815,
 'distance': np.float64(0.06411499625613173)}

In [ ]:
plot_interception(15, 0.5, "test_prediction_06.json", """Initial Position:  -0.870668 0.347448 3.36893
Initial Velocity:  2.89392 -2.68697 -3.34719
Acceleration -0.583966 10.9964 1.39556
Interception point: 0.503298 0.378516 1.86978""")

Interception point: (0.503, 0.379, 1.870) at t=0.500s
Closest data point: (0.553, 0.361, 1.813) at t=0.501s
Time difference: 0.001s
Spatial distance: 0.078m


{'interception_point': (np.float64(0.50329625),
  np.float64(0.3785129999999999),
  np.float64(1.8697800000000002)),
 'closest_point': (0.5533701171875, 0.3613311462402344, 1.8131607666015626),
 'interception_time': 0.5,
 'closest_time': 0.5014761599995836,
 'distance': np.float64(0.0775135205261644)}

In [ ]:
plot_interception(15, 0.5, "test_prediction_07.json", """Initial Position:  -0.860594 0.265691 2.87104
Initial Velocity:  2.77148 -2.13447 -3.02164
Acceleration -0.583966 10.9964 1.39556
Interception point: 0.452151 0.573007 1.53466""")

Interception point: (0.452, 0.573, 1.535) at t=0.500s
Closest data point: (0.391, 0.379, 1.597) at t=0.453s
Time difference: 0.047s
Spatial distance: 0.213m
Time to intercept: 0.317ms


{'interception_point': (np.float64(0.45215025),
  np.float64(0.573006),
  np.float64(1.5346649999999997)),
 'closest_point': (0.390579345703125, 0.3790950012207031, 1.5968968505859376),
 'interception_time': 0.5,
 'closest_time': 0.45255119200010085,
 'distance': np.float64(0.21275632759302346)}

In [ ]:
plot_interception(15, 0.5, "test_prediction_08.json", """Initial Position:  -0.232482 0.0819708 3.40825
Initial Velocity:  0.827777 -3.62155 -4.5221
Acceleration -0.583966 10.9964 1.39556
Interception point: 0.108411 -0.354251 1.32164""")

Interception point: (0.108, -0.354, 1.322) at t=0.500s
Closest data point: (0.107, -0.364, 1.413) at t=0.501s
Time difference: 0.001s
Spatial distance: 0.092m
Time to intercept: 0.280ms


{'interception_point': (np.float64(0.10841075),
  np.float64(-0.3542542000000002),
  np.float64(1.3216449999999997)),
 'closest_point': (0.10653827667236328,
  -0.3640301513671875,
  1.4130740966796875),
 'interception_time': 0.5,
 'closest_time': 0.5014748639987374,
 'distance': np.float64(0.0919693160850945)}

In [8]:
plot_interception(15, 0.5, "test_prediction_09.json", """Initial Position:  -0.490083 0.205492 3.4226
Initial Velocity:  1.45071 -3.42861 -4.22693
Acceleration -0.583966 10.9964 1.39556
Interception point: 0.162276 -0.134262 1.48358""")

Interception point: (0.162, -0.134, 1.484) at t=0.500s
Closest data point: (0.153, -0.142, 1.514) at t=0.501s
Time difference: 0.001s
Spatial distance: 0.033m
Time to intercept: 0.317ms


{'interception_point': (np.float64(0.16227624999999998),
  np.float64(-0.13426300000000002),
  np.float64(1.48358)),
 'closest_point': (0.15263525390625, -0.14224606323242187, 1.5143787841796874),
 'interception_time': 0.5,
 'closest_time': 0.5014758640008949,
 'distance': np.float64(0.033245198317945375)}

In [25]:
plot_interception(15, 0.5, "test_prediction_10.json", """Initial Position:  -0.525748 0.446621 3.52502
Initial Velocity:  1.22828 -3.36771 -4.16818
Acceleration -0.583966 10.9964 1.39556
Interception point: 0.0153947 0.137317 1.61538""")

Interception point: (0.015, 0.137, 1.615) at t=0.500s
Closest data point: (0.010, 0.085, 1.634) at t=0.498s
Time difference: 0.002s
Spatial distance: 0.056m
Time to intercept: 0.394ms


{'interception_point': (np.float64(0.015396250000000028),
  np.float64(0.13731599999999977),
  np.float64(1.6153749999999998)),
 'closest_point': (0.010147040367126464,
  0.08469021606445312,
  1.633942626953125),
 'interception_time': 0.5,
 'closest_time': 0.4981910000000198,
 'distance': np.float64(0.056051620023609604)}

In [26]:
plot_interception(15, 0.5, "test_prediction_11.json", """Initial Position:  -0.472776 -0.0297388 2.67505
Initial Velocity:  1.30475 -0.977797 -3.75667
Acceleration -0.509649 10.5443 2.40276
Interception point: 0.11589 0.799405 1.09706""")

Interception point: (0.116, 0.799, 1.097) at t=0.500s
Closest data point: (0.014, 0.417, 1.407) at t=0.404s
Time difference: 0.096s
Spatial distance: 0.503m
Time to intercept: 0.317ms


{'interception_point': (np.float64(0.11589287500000006),
  np.float64(0.7994002),
  np.float64(1.09706)),
 'closest_point': (0.01381160068511963, 0.41659896850585937, 1.40654931640625),
 'interception_time': 0.5,
 'closest_time': 0.40362591999996766,
 'distance': np.float64(0.5027335341597857)}